# Install

In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install datasets

In [ ]:
!pip3 install hydra-core

In [ ]:
!pip3 install wandb

In [ ]:
!pip3 install accelerate

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Few-shot learning

In [7]:
from hydra import compose, initialize

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [16]:
source_path="drive/MyDrive/Hackathon/src/training"

In [ ]:
import sys
sys.path.append(source_path)

from dataloader import load

In [18]:
with initialize(config_path=source_path):
    cfg = compose(config_name="config")

<ipython-input-18-6f879129a58a>:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=source_path):


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(**cfg.PATH.tokenizer_config)

In [21]:
train, test = load(tokenizer=tokenizer, **cfg.DATASETS)

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e83bf839bf6ac51f/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/22074 [00:00<?, ?ba/s]

  0%|          | 0/700 [00:00<?, ?ba/s]

  0%|          | 0/453 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

In [24]:
device = torch.device("cuda")

In [25]:
device

device(type='cuda')

In [26]:
print(tokenizer.decode(train[32]['input_ids'], skip_special_tokens=True))

당신의 성격:
나는 여자이다.
나는 30대이다.
나는 간호학과를 전공했다
나는 새벽에 산책한다
나는 김치찌개를 좋아한다

speaker1 이전 대화 요약: 
speaker2 이전 대화 요약: 

대화 요약:
speaker1: 나는 30대 여성이다
speaker2: 나는 40대 여성이다
speaker1: 나는 간호학과를 전공했다. 나는 병원에서 일한다.

다음 내용에 답하세요: 그러시군요. 저는 평택에 거주하고 있는데 이곳의 작은 회사에서 회계일을 하고 있어요.

답변: 회계일을 하시는군요. 저는 새벽에 산책하는 취미가 있어요. 취미 있으신가요?


In [27]:
text = """당신의 성격:
나는 여자이다.
나는 30대이다.
나는 간호학과를 전공했다
나는 새벽에 산책한다
나는 김치찌개를 좋아한다

speaker1 이전 대화 요약: 
speaker2 이전 대화 요약: 

대화 요약:
speaker1: 나는 30대 여성이다
speaker2: 나는 40대 여성이다
speaker1: 나는 간호학과를 전공했다. 나는 병원에서 일한다.

다음 내용에 답하세요: 그러시군요. 저는 평택에 거주하고 있는데 이곳의 작은 회사에서 회계일을 하고 있어요.

답변:"""

encoded = tokenizer(text)

sample = {k: torch.tensor([v]).to(device) for k, v in encoded.items()}

In [28]:
model = AutoModelForCausalLM.from_pretrained(**cfg.PATH.model_config).to(device).eval()

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

In [29]:
# 뭐야 왜 에러남
with torch.no_grad():
    pred = model.generate(
        **sample,
        penalty_alpha=0.6,
        top_k=5,
        max_length=512,
        eos_token_id=tokenizer.eos_token_id,
    )

In [30]:
print(tokenizer.decode(pred[0], skip_special_tokens=True))

당신의 성격:
나는 여자이다.
나는 30대이다.
나는 간호학과를 전공했다
나는 새벽에 산책한다
나는 김치찌개를 좋아한다

speaker1 이전 대화 요약: 
speaker2 이전 대화 요약: 

대화 요약:
speaker1: 나는 30대 여성이다
speaker2: 나는 40대 여성이다
speaker1: 나는 간호학과를 전공했다. 나는 병원에서 일한다.

다음 내용에 답하세요: 그러시군요. 저는 평택에 거주하고 있는데 이곳의 작은 회사에서 회계일을 하고 있어요.

답변: #
